In [1]:
from mlflow.tracking import MlflowClient


In [ ]:
MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"

In [5]:


client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)
client.create_experiment(name="my-cool-experiment")


'2'

In [7]:
from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids='1',
    filter_string="metrics.rmse < 7",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=["metrics.rmse ASC"]
)

In [8]:
for run in runs:
    print(f"run id: {run.info.run_id}, rmse: {run.data.metrics['rmse']:.4f}")

run id: 06cf19d4a5974699aea1eb11675a8349, rmse: 5.1180
run id: 6484504d02be46958097dc0be27b9f9a, rmse: 5.1206
run id: e7d8c1b2792d4fc09452bddb6d15e7c7, rmse: 5.1248
run id: 3f68adb10d7b4dceaea67821a6bb75a0, rmse: 5.1275
run id: c4d446b97f684072b532e9627d5569e6, rmse: 5.1313


In [9]:
import mlflow

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [10]:
run_id = "06cf19d4a5974699aea1eb11675a8349"
model_uri = f"runs:/{run_id}/model"
mlflow.register_model(model_uri=model_uri, name="nyc-taxi-reg")

Successfully registered model 'nyc-taxi-reg'.
Created version '1' of model 'nyc-taxi-reg'.


<ModelVersion: aliases=[], creation_timestamp=1719821344747, current_stage='None', description=None, last_updated_timestamp=1719821344747, name='nyc-taxi-reg', run_id='06cf19d4a5974699aea1eb11675a8349', run_link=None, source='/workspaces/MLOps-zoomcamp/02-experiment-tracking/mlruns/1/06cf19d4a5974699aea1eb11675a8349/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=1>

In [12]:
model_name = "nyc-taxi-reg"
latest_versions = client.get_latest_versions(name=model_name)

for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}")

version: 1, stage: None


/tmp/ipykernel_68005/4128852690.py:2: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  latest_versions = client.get_latest_versions(name=model_name)


In [16]:
model_version = 1
new_stage = "Staging"
client.transition_model_version_stage(
    name=model_name,
    version=model_version,
    stage=new_stage,
    archive_existing_versions=False
)

/tmp/ipykernel_68005/1600074043.py:3: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


<ModelVersion: aliases=[], creation_timestamp=1719821344747, current_stage='Staging', description=None, last_updated_timestamp=1719821503877, name='nyc-taxi-reg', run_id='06cf19d4a5974699aea1eb11675a8349', run_link=None, source='/workspaces/MLOps-zoomcamp/02-experiment-tracking/mlruns/1/06cf19d4a5974699aea1eb11675a8349/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=1>

In [17]:
from datetime import datetime

date = datetime.today().date()
client.update_model_version(
    name=model_name,
    version=model_version,
    description=f"The model version {model_version} was transitioned to {new_stage} on {date}"
)

<ModelVersion: aliases=[], creation_timestamp=1719821344747, current_stage='Staging', description='The model version 1 was transitioned to Staging on 2024-07-01', last_updated_timestamp=1719821540668, name='nyc-taxi-reg', run_id='06cf19d4a5974699aea1eb11675a8349', run_link=None, source='/workspaces/MLOps-zoomcamp/02-experiment-tracking/mlruns/1/06cf19d4a5974699aea1eb11675a8349/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=1>

In [23]:
client.transition_model_version_stage(
    name=model_name,
    version=1,
    stage="Production",
    archive_existing_versions=True
)

/tmp/ipykernel_68005/1316468422.py:1: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


<ModelVersion: aliases=[], creation_timestamp=1719821344747, current_stage='Production', description='The model version 1 was transitioned to Staging on 2024-07-01', last_updated_timestamp=1719822307365, name='nyc-taxi-reg', run_id='06cf19d4a5974699aea1eb11675a8349', run_link=None, source='/workspaces/MLOps-zoomcamp/02-experiment-tracking/mlruns/1/06cf19d4a5974699aea1eb11675a8349/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=1>